In [206]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [207]:
import pandas as pd
import numpy as np

Data collection-
Loading Dataset:






In [208]:
df = pd.read_csv("/content/uci-ml-phishing-dataset.csv")

In [209]:
df.head()

,id,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,1,-1,1,1,1,-1,-1,-1,-1,-1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,2,1,1,1,1,1,-1,0,1,-1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,3,1,0,1,1,1,-1,-1,-1,-1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,4,1,0,1,1,1,-1,-1,-1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,5,1,0,-1,1,1,-1,1,1,-1,...,-1,1,-1,-1,0,-1,1,1,1,1


In [210]:
df.shape

(11055, 32)

In [211]:
df['Result'].value_counts()

 1    6157
-1    4898
Name: Result, dtype: int64

In [212]:
df.columns

Index(['id', 'having_IP_Address', 'URL_Length', 'Shortining_Service',
       'having_At_Symbol', 'double_slash_redirecting', 'Prefix_Suffix',
       'having_Sub_Domain', 'SSLfinal_State', 'Domain_registeration_length',
       'Favicon', 'port', 'HTTPS_token', 'Request_URL', 'URL_of_Anchor',
       'Links_in_tags', 'SFH', 'Submitting_to_email', 'Abnormal_URL',
       'Redirect', 'on_mouseover', 'RightClick', 'popUpWidnow', 'Iframe',
       'age_of_domain', 'DNSRecord', 'web_traffic', 'Page_Rank',
       'Google_Index', 'Links_pointing_to_page', 'Statistical_report',
       'Result'],
      dtype='object')

Feature extraction-Finding correlation-coefficient between features

In [213]:
col_corr = set()
def correlation(dataset, threshold):
  corr_matrix = dataset.corr()
  for i in range(len(corr_matrix.columns)):
      for j in range(i):
          if (abs(corr_matrix.iloc[i, j]) >= threshold) and (corr_matrix.columns[j] not in col_corr):
              colname = corr_matrix.columns[i]
              col_corr.add(colname)

In [214]:
correlation(df, 0.85)
col_corr

{'popUpWidnow'}

In [215]:
weak_col_corr = set()
def weakcorrelation(dataset, threshold):
  corr_matrix = dataset.corr()
  idx = 0
  for feature in corr_matrix['Result']:
    if(feature < threshold):
      weak_col_corr.add(corr_matrix.columns[idx])
    idx += 1
  
 # dropping features with correlation less than 0.01 
weakcorrelation(df, 0.01)
print(weak_col_corr)

{'id', 'Favicon', 'Redirect', 'double_slash_redirecting', 'Domain_registeration_length', 'popUpWidnow', 'Abnormal_URL', 'HTTPS_token', 'Iframe', 'Shortining_Service'}


In [216]:
del_col = col_corr.union(weak_col_corr)
del_col

{'Abnormal_URL',
 'Domain_registeration_length',
 'Favicon',
 'HTTPS_token',
 'Iframe',
 'Redirect',
 'Shortining_Service',
 'double_slash_redirecting',
 'id',
 'popUpWidnow'}

In [217]:
df.isnull().sum()

id                             0
having_IP_Address              0
URL_Length                     0
Shortining_Service             0
having_At_Symbol               0
double_slash_redirecting       0
Prefix_Suffix                  0
having_Sub_Domain              0
SSLfinal_State                 0
Domain_registeration_length    0
Favicon                        0
port                           0
HTTPS_token                    0
Request_URL                    0
URL_of_Anchor                  0
Links_in_tags                  0
SFH                            0
Submitting_to_email            0
Abnormal_URL                   0
Redirect                       0
on_mouseover                   0
RightClick                     0
popUpWidnow                    0
Iframe                         0
age_of_domain                  0
DNSRecord                      0
web_traffic                    0
Page_Rank                      0
Google_Index                   0
Links_pointing_to_page         0
Statistica

Test-Train split

In [218]:
from sklearn.model_selection import train_test_split 
from sklearn import metrics

In [219]:
x = df.iloc[: , :-1]
y = df.iloc[:, -1:]

In [220]:
x

,id,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,...,RightClick,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report
0,1,-1,1,1,1,-1,-1,-1,-1,-1,...,1,1,1,-1,-1,-1,-1,1,1,-1
1,2,1,1,1,1,1,-1,0,1,-1,...,1,1,1,-1,-1,0,-1,1,1,1
2,3,1,0,1,1,1,-1,-1,-1,-1,...,1,1,1,1,-1,1,-1,1,0,-1
3,4,1,0,1,1,1,-1,-1,-1,1,...,1,1,1,-1,-1,1,-1,1,-1,1
4,5,1,0,-1,1,1,-1,1,1,-1,...,1,-1,1,-1,-1,0,-1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11050,11051,1,-1,1,-1,1,1,1,1,-1,...,-1,-1,-1,1,1,-1,-1,1,1,1
11051,11052,-1,1,1,-1,-1,-1,1,-1,-1,...,1,-1,1,1,1,1,1,1,-1,1
11052,11053,1,-1,1,1,1,-1,1,-1,-1,...,1,1,1,1,1,1,-1,1,0,1
11053,11054,-1,-1,1,1,1,-1,-1,-1,1,...,1,-1,1,1,1,1,-1,1,1,1


In [221]:
y

,Result
0,-1
1,-1
2,-1
3,-1
4,1
...,...
11050,1
11051,-1
11052,-1
11053,-1


In [222]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1)

Model Training using ml algorithms

Decision Tree Classifier

In [223]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc = dtc.fit(x_train,y_train)
predict_result = dtc.predict(x_test)


In [224]:
dtc.score(x_train,y_train)

1.0

In [225]:
print("Accuracy:",metrics.accuracy_score(y_test, predict_result))

Accuracy: 0.9511533242876526


In [226]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, predict_result))

              precision    recall  f1-score   support

          -1       0.95      0.94      0.95       996
           1       0.95      0.96      0.96      1215

    accuracy                           0.95      2211
   macro avg       0.95      0.95      0.95      2211
weighted avg       0.95      0.95      0.95      2211



In [227]:
confusion_matrix(y_test, predict_result)

array([[ 932,   64],
       [  44, 1171]])

Random forest classifier

In [228]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(x_train, y_train)
predict_result = rfc.predict(x_test)

<ipython-input-228-0196aba91506>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(x_train, y_train)


In [229]:
print("Accuracy:",metrics.accuracy_score(y_test, predict_result))

Accuracy: 0.9728629579375848


In [230]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, predict_result))

              precision    recall  f1-score   support

          -1       0.97      0.96      0.97       996
           1       0.97      0.98      0.98      1215

    accuracy                           0.97      2211
   macro avg       0.97      0.97      0.97      2211
weighted avg       0.97      0.97      0.97      2211



In [231]:
confusion_matrix(y_test, predict_result)

array([[ 961,   35],
       [  25, 1190]])

Support Vector Machine


In [232]:
from sklearn.svm import SVC
svc= SVC(C= .1, gamma= 1, kernel='sigmoid', random_state=42)
svc.fit(x_train, y_train)
predict_result = svc.predict(x_test)



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [233]:
print("Accuracy:",metrics.accuracy_score(y_test, predict_result))

Accuracy: 0.5495251017639078


In [234]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, predict_result))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       996
           1       0.55      1.00      0.71      1215

    accuracy                           0.55      2211
   macro avg       0.27      0.50      0.35      2211
weighted avg       0.30      0.55      0.39      2211



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [235]:
confusion_matrix(y_test, predict_result)

array([[   0,  996],
       [   0, 1215]])

K-nearest neighbors

In [236]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier(n_neighbors=1)
knn.fit(x_train,y_train)
predict_result = knn.predict(x_test)

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [237]:
print("Accuracy:",metrics.accuracy_score(y_test, predict_result))

Accuracy: 0.6879240162822252


In [238]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, predict_result))

              precision    recall  f1-score   support

          -1       0.65      0.67      0.66       996
           1       0.72      0.70      0.71      1215

    accuracy                           0.69      2211
   macro avg       0.69      0.69      0.69      2211
weighted avg       0.69      0.69      0.69      2211



In [239]:
confusion_matrix(y_test, predict_result)

array([[670, 326],
       [364, 851]])

Logistic Regression 

In [240]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=1)
lr.fit(x_train,y_train)
predict_result = lr.predict(x_test)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [241]:
print("Accuracy:",metrics.accuracy_score(y_test, predict_result))

Accuracy: 0.919041157847128


In [242]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, predict_result))

              precision    recall  f1-score   support

          -1       0.92      0.90      0.91       996
           1       0.92      0.93      0.93      1215

    accuracy                           0.92      2211
   macro avg       0.92      0.92      0.92      2211
weighted avg       0.92      0.92      0.92      2211



In [243]:
confusion_matrix(y_test, predict_result)

array([[ 898,   98],
       [  81, 1134]])

Naive Bayes

In [244]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, CategoricalNB, BernoulliNB, ComplementNB
nb = BernoulliNB()
nb.fit(x_train,y_train)
predict_result = nb.predict(x_test)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [245]:
print("Accuracy:",metrics.accuracy_score(y_test, predict_result))

Accuracy: 0.9077340569877883


In [246]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, predict_result))

              precision    recall  f1-score   support

          -1       0.90      0.89      0.90       996
           1       0.91      0.92      0.92      1215

    accuracy                           0.91      2211
   macro avg       0.91      0.91      0.91      2211
weighted avg       0.91      0.91      0.91      2211



In [247]:
confusion_matrix(y_test, predict_result)

array([[ 889,  107],
       [  97, 1118]])

AdaBoostClassifier

In [248]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier(n_estimators=100, random_state=42)
abc.fit(x_train,y_train)
predict_result = abc.predict(x_test)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [249]:
print("Accuracy:",metrics.accuracy_score(y_test, predict_result))

Accuracy: 0.932157394843962


In [250]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, predict_result))

              precision    recall  f1-score   support

          -1       0.94      0.90      0.92       996
           1       0.92      0.96      0.94      1215

    accuracy                           0.93      2211
   macro avg       0.93      0.93      0.93      2211
weighted avg       0.93      0.93      0.93      2211



In [251]:
confusion_matrix(y_test, predict_result)

array([[ 900,   96],
       [  54, 1161]])

Hybrid Ensembler

In [252]:
from sklearn.ensemble import VotingClassifier

In [253]:
model_1 = RandomForestClassifier(n_estimators=100)
model_2 = KNeighborsClassifier(n_neighbors=1)
model_3 = LogisticRegression(random_state=0)
model_4 = BernoulliNB()
model_5 = DecisionTreeClassifier()
ensemble = VotingClassifier(estimators=[('RandomForest', model_1), ('KNN', model_2), ('LogisticRegression', model_3), ('NaiveBayes', model_4),
                                        ('DT', model_5)], voting='hard')

In [254]:
ensemble.fit(x_train, y_train)
predict_result = ensemble.predict(x_test)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear

In [255]:
print("Accuracy:",metrics.accuracy_score(y_test, predict_result))

Accuracy: 0.9479873360470376


In [256]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, predict_result))

              precision    recall  f1-score   support

          -1       0.95      0.94      0.94       996
           1       0.95      0.96      0.95      1215

    accuracy                           0.95      2211
   macro avg       0.95      0.95      0.95      2211
weighted avg       0.95      0.95      0.95      2211



In [257]:
confusion_matrix(y_test, predict_result)

array([[ 933,   63],
       [  52, 1163]])